In [1]:
from crewai import Agent, Task, Crew, Process
from datetime import datetime
import os
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Setup API key
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
# SETUP api key
os.getenv("OPENAI_API_KEY")
os.getenv("SERPER_API_KEY")

'5c3b65bbe9ff6211fdda9025b9b2d0c8ae63af4f'

In [4]:
from langchain_openai import ChatOpenAI

# Initialize the OpenAI GPT-4 model
llm = ChatOpenAI(model="gpt-3.5-turbo")

### Save markdown file

In [5]:
def save_markdown_file(task_output):
    today_date = datetime.now().strftime("%Y-%m-%d")
    filename = f"{today_date}.md"
    with open(filename, "w") as file:
        file.write(task_output.result)
    print(f"Newletter saved as {filename}")

In [6]:
from crewai_tools import SerperDevTool

search_tool = SerperDevTool(name="Search the internet")

### Settings up agents

In [7]:
class AINewsLetterAgent:
    def editor_agent(self):
        return Agent(
            role='Editor',
            goal='Oversee the creation of the AI Newsletter',
            backstory=
            """ 
            With a keen eye for detail and a passion for storytelling,
            you ensure that the newsletter
            not only informs but also engages and inspires the readers.
            """,
            allow_delegation=True,
            verbose=True,
            max_iter=1
        )
    
    def news_fetcher_agent(self):
        return Agent(
            role = "NewsFetcher",
            goal = "Fetch the top AI news stories for the day",
            backstory=
            """
                As a digital sleuth, you scour the internet for the latest and most impactful developments
                in the world of AI, ensuring that our readers are alwaysin the know.
            """,
            tools=[search_tool],
            verbose = True,
            allow_delegation=True
        )
    
    def news_analyzer_agent(self):
        return Agent(
            role = "NewsAnalyzer",
            goal = "Analyze each news story and generate a detailed markdown summary",
            backstory="""
                With a critical eye and a knack for distilling complex information, you analyses of AI news stories, making them accessible and engagin for our audience.
            """,
            tools=[search_tool],
            verbose = True,
            allow_delegation=True
        )
    def newsletter_comiler_agent(self):
        return Agent(
            role = "NewsletterCompiler",
            goal = "Compile the analyzed news stories into a final newsletter format",
            backstory="""
                As a final architect of the newsletter, you meticulously arrange and ensuing a coherent and visually appealing presentations that captivates out reader newsletter format guidelines and maintain consistency throughout.
            """,
            tools=[search_tool],
            verbose = True,
            allow_delegation=True
        )

### Settings up tasks

In [8]:
class AINewsletterTask():
    def fetch_news_task(self, agent):
        return Task(
            description=f"Fetch top AI news stories from the past 24 hours. The current time is {datetime.now()}",
            agent = agent,
            async_execution=True,
            expected_output="""
                A list of top AI news story titles, URLs, and a brief summary for each story from the past 24 hours.
                Example Output:
                [
                    {
                        'title': 'AI takes spotlight in Super Bowl commercials',
                        'url': 'https://example.com/story1',
                        'summary': 'AI made a splash in this year\'s Super Bowl commercials...',
                        {{...}}
                    } 
                ]
            """
        )
    def analyze_news_task(self, agent, context):
        return Task(
            description="Analyse each news story and ensure there are at least 5 well-formatted articles",
            agent = agent,
            async_execution=True,
            context= context,
            expected_output=
            """
                A markdown-formatted analysis for each news story, including a rundown, and a "Why it matters" section. There should be at least 5 articles, each following the proper format.
                Examp Output:
                '## AI takes spotlight in Super Bowl commercials \n\n
                **The Rundown:
                ** AI made a splash in this year
                \'s Super Bowl Commercials..\n\n
                **The details:**\n\n
                - Microsoft\'s Copilot spot showcased its AI assistant...\n\n
                **Why it matter:** While AI-related ads have been rampant over the last year, its Super Bowl present
            """,
        )
    
    def compile_newsletter_task(self, agent, context, callback_function):
        return Task(
            description="Compile the newsletter",
            agent=agent,
            context=context,
            expected_output = 
            """
            A complete newsletter in markdown format. with a consistent style and layout.
            Example Output:
            '# Top stories in AI today:\\n\\n
            - AI takes spotlight in Super Bowl commercials\\n
            - AItman seeks TRILLIONS for global AI chip initiative\\n\\n

            ## AI takes spotlight in Super Bowl commercials\\n\n
            **The Rundown:** AI made a splash in this year\'s Super Bowl commercials...\\n\\n
            **The details:**...\\n\\n
            **Why it matters::**..\\n\\n
            ## Altman seeks TRILLIONS for global AI chip initiative\\n\\n
            **The Rundown:** OpenAI CEO Altman is reportedly angling to raise TRILLIONS of dollors...\\n\\n
            **The details:**...\\n\\n
            **Why is matters::**...\\n\\n
            """,
            callback=callback_function
        )

### Settings up tools

In [9]:
agents = AINewsLetterAgent()
tasks = AINewsletterTask()

In [10]:
# ALL AGENTs
editor = agents.editor_agent()
news_fetcher = agents.news_fetcher_agent()
news_analyzer = agents.news_analyzer_agent()
newsletter_compiler = agents.newsletter_comiler_agent()

In [11]:
# ALL TASKs
fetch_news_task = tasks.fetch_news_task(news_fetcher)
analyzed_news_task = tasks.analyze_news_task(news_analyzer, [fetch_news_task])
compiled_newsletter_task = tasks.compile_newsletter_task(newsletter_compiler, [analyzed_news_task], callback_function=save_markdown_file)

### Settings up environment

In [12]:
crew = Crew(
    agents=[editor, news_fetcher, news_analyzer, newsletter_compiler],
    tasks=[fetch_news_task, analyzed_news_task, compiled_newsletter_task],
    process=Process.sequential,
)

### Run the crew

In [13]:
# Kick of the crew
results = crew.kickoff()
print("Crew work results:")
print(results)



> Entering new CrewAgentExecutor chain...
I need to find the top AI news stories from the past 24 hours. To do this, I should use the search the internet tool to find the most recent and relevant news articles.

Action: 
Search the internet

Action Input: 
{
  "search_query": "top AI news stories 2024-04-15 to 2024-04-16"
} 


Search results: Title: [FULL] AI HEADLINE NEWS 16:00 (2024-04-15) - YouTube
Link: https://www.youtube.com/watch?v=Kh6_ZM6ZMOY
Snippet: ... highest ranking official, Zhao Leji on ... Report. Comments. thumbnail-image. Add a comment ...
---
Title: Microsoft Makes High-Stakes Play in Tech Cold War With Emirati A.I. ...
Link: https://www.nytimes.com/2024/04/16/technology/microsoft-g42-uae-ai.html
Snippet: Microsoft Makes High-Stakes Play in Tech Cold War With Emirati A.I. Deal. Microsoft said it would invest $1.5 billion in G42, an Emirati company ...
---
Title: "Top secret" is no longer the key to good intel in an AI world: report
Link: https://www.axios.com/2024/